<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/Emotionline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emitionline 데이터셋

In [1]:
! tar -xvzf /content/drive/My\ Drive/Datasets/Emotionlines/EmotionLines_friends_annotation.tar.gz

EmotionLines/Friends/friends_train.json
EmotionLines/Friends/friends_dev.json
EmotionLines/Friends/friends_test.json
EmotionLines/README


In [68]:
import pandas as pd
import json

In [69]:
with open('/content/EmotionLines/Friends/friends_train.json', encoding='utf-8', mode='r') as f:
  array = json.load(f)

data =[]
for i in range(len(array)):
  data += array[i]
print(f"Total length of data : {len(data)}")
df = pd.DataFrame(data)

Total length of data : 10561


In [70]:
df

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030
...,...,...,...,...
10556,Chandler,You or me?,neutral,3000011
10557,Ross,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral,2100011
10558,Joey,"You guys are messing with me, right?",surprise,0000050
10559,All,Yeah.,neutral,4000010


In [71]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [72]:
# change the value to dtype: Category
df['emotion'] = pd.Categorical(df['emotion'])

# cat(egory).codes print the code of category
emotion_codes = df.emotion.cat.codes

In [73]:
emotion_codes

0        4
1        4
2        4
3        4
4        7
        ..
10556    4
10557    5
10558    7
10559    4
10560    5
Length: 10561, dtype: int8

In [74]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [75]:
for i, utter in enumerate(df.utterance):
  if i < 5:
    print(utter)
  else:
    break

also I was the point person on my companys transition from the KL-5 to GR-6 system.
You mustve had your hands full.
That I did. That I did.
So lets talk a little bit about your duties.
My duties?  All right.


# make data



make the dataset to tensor

In [76]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

In [77]:
x_train = df.pop('utterance')
y_train = emotion_codes

In [78]:
x_train.values

array(['also I was the point person on my company\x92s transition from the KL-5 to GR-6 system.',
       'You must\x92ve had your hands full.', 'That I did. That I did.',
       ..., 'You guys are messing with me, right?', 'Yeah.',
       'That was a good one. For a second there, I was like, "whoa."'],
      dtype=object)

In [79]:
y_onehot = tf.one_hot(y_train, depth=8)
y_onehot.shape

TensorShape([10561, 8])

이 이하는 실행 안 해도 됨

In [59]:
dataset = tf.data.Dataset.from_tensor_slices(x_train)

In [15]:
for t in dataset.take(5):
  print(t)

tf.Tensor(b'also I was the point person on my company\xc2\x92s transition from the KL-5 to GR-6 system.', shape=(), dtype=string)
tf.Tensor(b'You must\xc2\x92ve had your hands full.', shape=(), dtype=string)
tf.Tensor(b'That I did. That I did.', shape=(), dtype=string)
tf.Tensor(b'So let\xc2\x92s talk a little bit about your duties.', shape=(), dtype=string)
tf.Tensor(b'My duties?  All right.', shape=(), dtype=string)


In [16]:
train_data = dataset.shuffle(len(df))
for t in train_data.take(5):
  print(t)

tf.Tensor(b'I uh, went to a bar. And then I just uh, just walked around for a while.', shape=(), dtype=string)
tf.Tensor(b'Okay.', shape=(), dtype=string)
tf.Tensor(b'Oh.', shape=(), dtype=string)
tf.Tensor(b'Yeah, I\xc2\x92m just, I\xc2\x92m just in town for a conference. Umm,', shape=(), dtype=string)
tf.Tensor(b'What about these? These look the same?', shape=(), dtype=string)


# Tokenizer

In [17]:
tokenizer = K.preprocessing.text.Tokenizer(num_words = 4000, char_level=False)

In [18]:
tokenizer.fit_on_texts(x_train.values)

In [60]:
x_in = tokenizer.texts_to_sequences(x_train)

In [61]:
x_in = tf.keras.preprocessing.sequence.pad_sequences(x_in)
print(x_in)

[[   0    0    0 ... 3069  968  969]
 [   0    0    0 ...   39  760  882]
 [   0    0    0 ...    9    1   71]
 ...
 [   0    0    0 ...   37   14   35]
 [   0    0    0 ...    0    0   17]
 [   0    0    0 ...   33   41  184]]


In [63]:
x_in.shape

(10561, 68)

# Embedding

In [22]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, ReLU, Softmax

## Hyperparameter

In [23]:
INPUT_DIM = 4000
EMBED_DIM = 1000
INPUT_LEN = 68
RNN_DEPTH = 128
FIRST_DENSE = 64
OUTPUT_LEN = 8
BATCH_SIZE = 100
EPOCHS = 10
SHUFFLE = True

Model Create

In [24]:
model = K.Sequential()
model.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
model.add(Flatten())
model.add(Dense(FIRST_DENSE))
model.add(ReLU())
model.add(Dense(OUTPUT_LEN))
model.add(Softmax())

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 68, 1000)          4000000   
_________________________________________________________________
flatten (Flatten)            (None, 68000)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                4352064   
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax (Softmax)            (None, 8)                 0         
Total params: 8,352,584
Trainable params: 8,352,584
Non-trainable params: 0
______________________________________________

In [25]:
model.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [26]:
model.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

Epoch 1/10
106/106 [==============================] - 7s 62ms/step - loss: 1.6018
Epoch 2/10
106/106 [==============================] - 7s 62ms/step - loss: 1.2653
Epoch 3/10
106/106 [==============================] - 6s 61ms/step - loss: 0.9284
Epoch 4/10
106/106 [==============================] - 7s 62ms/step - loss: 0.6862
Epoch 5/10
106/106 [==============================] - 6s 61ms/step - loss: 0.5377
Epoch 6/10
106/106 [==============================] - 7s 62ms/step - loss: 0.4481
Epoch 7/10
106/106 [==============================] - 7s 61ms/step - loss: 0.3885
Epoch 8/10
106/106 [==============================] - 6s 61ms/step - loss: 0.3549
Epoch 9/10
106/106 [==============================] - 7s 62ms/step - loss: 0.3316
Epoch 10/10
106/106 [==============================] - 7s 62ms/step - loss: 0.3099


# Model evaluation

In [27]:
def make_test_sample(text):
  test_data = tokenizer.texts_to_sequences(text)
  test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=68)
  print(test_data)
  print(test_data.shape)
  return test_data

In [28]:
test_data = make_test_sample(["Hello Hello I'm Jisoo"])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 191 191  52]]
(1, 68)


In [29]:
result = model.predict(test_data, batch_size=BATCH_SIZE)

In [30]:
answer = result.argmax()
print(emo_cat.categories[answer])

NameError: ignored

# Categories And codes

In [31]:
emo_cat = pd.Categorical(df['emotion'])
emo_cat.categories
emo_cat.codes

array([4, 4, 4, ..., 7, 4, 5], dtype=int8)

In [32]:
emo_df = pd.DataFrame(
    {"emotion" : emo_cat.categories}
)
emo_df

,emotion
0,anger
1,disgust
2,fear
3,joy
4,neutral
5,non-neutral
6,sadness
7,surprise


# Make RNN model

In [33]:
from tensorflow.keras.layers import SimpleRNN

In [34]:
rnnModel = K.Sequential()
rnnModel.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
rnnModel.add(SimpleRNN(RNN_DEPTH))
rnnModel.add(Flatten())
rnnModel.add(Dense(FIRST_DENSE))
rnnModel.add(ReLU())
rnnModel.add(Dense(OUTPUT_LEN))
rnnModel.add(Softmax())

rnnModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 68, 1000)          4000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               144512    
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 520       
_________________________________________________________________
softmax_1 (Softmax)          (None, 8)                

In [35]:
rnnModel.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [36]:
rnnModel.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

Epoch 1/10
106/106 [==============================] - 13s 124ms/step - loss: 1.5747
Epoch 2/10
106/106 [==============================] - 13s 121ms/step - loss: 1.2864
Epoch 3/10
106/106 [==============================] - 13s 122ms/step - loss: 0.8886
Epoch 4/10
106/106 [==============================] - 13s 122ms/step - loss: 0.6233
Epoch 5/10
106/106 [==============================] - 13s 123ms/step - loss: 0.4880
Epoch 6/10
106/106 [==============================] - 13s 122ms/step - loss: 0.4071
Epoch 7/10
106/106 [==============================] - 13s 123ms/step - loss: 0.3537
Epoch 8/10
106/106 [==============================] - 13s 123ms/step - loss: 0.3221
Epoch 9/10
106/106 [==============================] - 13s 122ms/step - loss: 0.3006
Epoch 10/10
106/106 [==============================] - 13s 120ms/step - loss: 0.2921


# Make Validation Generator

In [37]:
def validation_preprocessor(tokenizer):
  with open('/content/EmotionLines/Friends/friends_dev.json', encoding='utf-8', mode='r') as f:
    array = json.load(f)

  data =[]
  for i in range(len(array)):
    data += array[i]
  print(f"Total length of data : {len(data)}")
  df = pd.DataFrame(data)

    # change the value to dtype: Category
  df['emotion'] = pd.Categorical(df['emotion'])

  # cat(egory).codes print the code of category
  emotion_codes = df.emotion.cat.codes

  x_train = df.pop('utterance')
  y_train = emotion_codes
  y_onehot = tf.one_hot(y_train, depth=8)

  tokenizer.fit_on_texts(x_train.values)
  x_in = tokenizer.texts_to_sequences(x_train)
  x_in = tf.keras.preprocessing.sequence.pad_sequences(x_in, maxlen=68)

  return x_in, y_onehot

In [38]:
x_in, y_onehot = validation_preprocessor(tokenizer)

Total length of data : 1178


In [44]:
rnnModel.evaluate(x=x_in, y = y_onehot)

37/37 [==============================] - 1s 24ms/step - loss: 0.1509


0.15094460546970367

In [45]:
model.evaluate(x=x_in, y = y_onehot)

37/37 [==============================] - 0s 5ms/step - loss: 3.4863


3.4863157272338867

# Make Multiple RNN model

In [41]:
rnnModel2 = K.Sequential()
rnnModel2.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
rnnModel2.add(SimpleRNN(RNN_DEPTH, return_sequences=True))
rnnModel2.add(SimpleRNN(RNN_DEPTH))
rnnModel2.add(Flatten())
rnnModel2.add(Dense(FIRST_DENSE))
rnnModel2.add(ReLU())
rnnModel2.add(Dense(OUTPUT_LEN))
rnnModel2.add(Softmax())

rnnModel2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 68, 1000)          4000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 68, 128)           144512    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                

In [42]:
rnnModel2.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [48]:
rnnModel2.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

Epoch 1/10
12/12 [==============================] - 2s 159ms/step - loss: 1.7643
Epoch 2/10
12/12 [==============================] - 2s 174ms/step - loss: 1.6060
Epoch 3/10
12/12 [==============================] - 2s 163ms/step - loss: 1.3229
Epoch 4/10
12/12 [==============================] - 2s 167ms/step - loss: 0.9244
Epoch 5/10
12/12 [==============================] - 2s 163ms/step - loss: 0.5940
Epoch 6/10
12/12 [==============================] - 2s 161ms/step - loss: 0.4119
Epoch 7/10
12/12 [==============================] - 2s 163ms/step - loss: 0.2785
Epoch 8/10
12/12 [==============================] - 2s 168ms/step - loss: 0.2167
Epoch 9/10
12/12 [==============================] - 2s 164ms/step - loss: 0.1831
Epoch 10/10
12/12 [==============================] - 2s 163ms/step - loss: 0.1553


In [49]:
rnnModel2.evaluate(x=x_in, y = y_onehot,batch_size=BATCH_SIZE)

12/12 [==============================] - 0s 39ms/step - loss: 0.1374


0.13742001354694366

# Make LSTM Model

In [55]:
lstmModel = K.Sequential()
lstmModel.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
lstmModel.add(SimpleRNN(RNN_DEPTH, return_sequences=True))
lstmModel.add(SimpleRNN(RNN_DEPTH))
lstmModel.add(Flatten())
lstmModel.add(Dense(FIRST_DENSE))
lstmModel.add(ReLU())
lstmModel.add(Dense(OUTPUT_LEN))
lstmModel.add(Softmax())

lstmModel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 68, 1000)          4000000   
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 68, 128)           144512    
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 128)               32896     
_________________________________________________________________
flatten_4 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
re_lu_4 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 8)                

In [66]:
lstmModel.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [81]:
lstmModel.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

Epoch 1/10
106/106 [==============================] - 20s 192ms/step - loss: 0.2882
Epoch 2/10
106/106 [==============================] - 21s 194ms/step - loss: 0.2885
Epoch 3/10
106/106 [==============================] - 21s 194ms/step - loss: 0.2903
Epoch 4/10
106/106 [==============================] - 21s 194ms/step - loss: 0.2731
Epoch 5/10
106/106 [==============================] - 20s 190ms/step - loss: 0.2551
Epoch 6/10
106/106 [==============================] - 20s 193ms/step - loss: 0.2397
Epoch 7/10
106/106 [==============================] - 20s 190ms/step - loss: 0.2349
Epoch 8/10
106/106 [==============================] - 20s 186ms/step - loss: 0.2249
Epoch 9/10
106/106 [==============================] - 20s 188ms/step - loss: 0.2283
Epoch 10/10
106/106 [==============================] - 20s 188ms/step - loss: 0.2861
